In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
# from pandasql import sqldf

In [10]:
def preprocessing(df):  

  #drop duplicated columns
  drop_column=['description2', 'Unnamed: 0', 'link', 'author_y', 'title','url', 'title_1']
  df=df.drop(drop_column,axis=1)

  #renaming
  column_names=['author', 'date_recorded', 'views', 'likes', 'title',
       'description_1', 'duration_seg', 'date_released', 'keywords','description_2']
  df.columns=column_names

  #MODIFYING COLUMN: date_recorded
  df['date_recorded']= pd.to_datetime(df['date_recorded'], format='%B %Y')
  
  ##separate data into new Column
  list_months=[]
  list_years=[]
  for i in range(df.shape[0]):
      list_months.append(df['date_recorded'][i].month)
      list_years.append(df['date_recorded'][i].year)
  df['date_recorded_year']=list_years
  df['date_recorded_month']=list_months

  #mMODIFYING COLUMN: date_released
  column='date_released'
  df[column]= pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S')

  ##separate data into new Column
  list_months=[]
  list_years=[]
  list_hours=[]
  list_minutes=[]
  for i in range(df.shape[0]):
      list_months.append(df[column][i].month)
      list_years.append(df[column][i].year)
      list_hours.append(df[column][i].hour)
      list_minutes.append(df[column][i].minute)
  df[column+'_year']=list_years
  df[column+'_month']=list_months
  df[column+'_hour']=list_hours
  df[column+'_minute']=list_minutes

  #MODIFYING COLUMN: 'keywords'
  df_key=df.keywords
  i=0
  df_result=pd.DataFrame()
  ##transforming line into string
  for line in df_key:
    line=(str(line).replace("[","").replace("]","").split(','))
    new_line=[]
    ##removing additional spaces in words and converting the into lower case
    for word in line:
      word=word.lower().replace(' ', '')[1:-1]
      new_line.append(word)
    ##transforming line into string
    new_line=str(new_line).replace("[","").replace("]","")
    ##writting line into dataframe
    df_result.at[i,'keywords2']=new_line
    i=i+1
  
  df=pd.concat([df,df_result], axis=1)

  #drop initial columns
  drop_columns=['date_recorded','date_released', 'keywords']
  df=df.drop(drop_columns, axis=1)

  return(df)

In [11]:
def main():
  #load data
  raw_data=pd.read_csv('https://github.com/aaas24/code_library/raw/main/ted_talks/1_raw_data/final_raw_data.csv')
  df=raw_data.copy()

  #clean data
  df=preprocessing(df)

  return(df)
  


In [12]:
if __name__ == '__main__':
  main()

## Data Exploration

In [15]:
# missing data
((df
 .isna()
 .mean()
 *100)
 .pipe(lambda ser:ser[ser>0])
)

author    0.018382
dtype: float64

In [18]:
#converting keywords into dummy columns
df2=df.keywords2.str.get_dummies(',')
print(df.shape)

(5440, 349)


In [19]:
df.dtypes

 "alzheimer's"         int64
 ''                    int64
 '3dprinting'          int64
 'activism'            int64
 'addiction'           int64
                       ...  
 'work'                int64
 'work-lifebalance'    int64
 'writing'             int64
 'youth'               int64
'ted'                  int64
Length: 349, dtype: object

In [21]:
df=main()
df.head(5)

,author,views,likes,title,description_1,duration_seg,description_2,date_recorded_year,date_recorded_month,date_released_year,date_released_month,date_released_hour,date_released_minute,keywords2
0,Ozawa Bineshi Albert,404000,12000,Climate action needs new frontline leadership,"""We can't rely on those who created climate ch...",834,"""We can't rely on those who created climate ch...",2021,12,2022,2,9,41,"'ted', 'talks', 'climatechange', 'globalissues..."
1,Sydney Iaukea,214000,6400,The dark history of the overthrow of Hawaii,"""On January 16th, 1895, two men arrived at Lil...",0,"""On January 16th, 1895, two men arrived at Lil...",2022,2,2022,2,10,13,"'ted', 'talks', 'education', 'women', 'ted-ed'..."
2,Martin Reeves,412000,12000,Why play is essential for business,"""To thrive in today's competitive economy, you...",665,"""To thrive in today's competitive economy, you...",2021,9,2022,2,9,51,"'ted', 'talks', 'business', 'leadership', 'wor..."
3,James K. Thornton,427000,12000,Why is China appointing judges to combat clima...,"""Why is China appointing thousands of judges t...",695,"""Why is China appointing thousands of judges t...",2021,10,2022,2,9,45,"'ted', 'talks', 'climatechange', 'pollution', ..."
4,Mahendra Singhi,2400,72,Cement's carbon problem -- and 2 ways to fix it,"""Cement is vital to modernizing all kinds of i...",671,"""Cement is vital to modernizing all kinds of i...",2021,10,2022,2,9,38,"'ted', 'talks', 'climatechange', 'countdown', ..."


# Analysis

In [ ]:
# #what constitude a good video based on likes?
y=1000
df_graph=df.likes.apply(lambda x: round(x/y,0))
df_graph.describe()

In [ ]:
# #what constitude a good video based on views?
# y=1000
# df_graph=df.views.apply(lambda x: round(x/y,0))
# df_graph.describe()

### Top 10 Liked Videos

In [ ]:
# #build data: These are the 75% most liked videos
# y_var='likes'
# df_grap = (
#     (df.groupby(['title','author_x','views', 'keywords'])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index()
# df_grap=df_grap.drop('index', axis=1)
# df_grap = df_grap[df_grap[y_var] > 65000]
# df_grap

In [ ]:
# #This is taking the 75% percentile on liked videos
# #Modifiable variables
# y_var='likes'

# #build data
# df_grap = (
#     (df.groupby(['title_2','author','views'])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index()
# df_grap=df_grap.drop('index', axis=1)
# df_grap = df_grap[df_grap[y_var] > 65000]
# df_grap.head(30)

### Top 10 Authors

In [ ]:
# #Modifiable variables
# y_var='views'
# x_var1='author'
# x_var2='likes'

# #build data
# df_grap = (
#     (df.groupby([x_var1, x_var2])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index().head(10)
# df_grap=df_grap.drop('index', axis=1)
# df_grap

In [ ]:
# #visualize
# x_var='author'
# y_var='views'
# sns.set_theme(style="white")
# ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h")

In [ ]:
# #visualize
# x_var='author'
# y_var='likes'
# sns.set_theme(style="white")
# ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h", palette="Blues_d")

In [ ]:
#from pandasql import sqldf
# q = """
# SELECT
# `title`
# FROM
# df
# LIMIT 5
# """
# print(sqldf(q))

Areas of improvements:
    
    1) More information on the authors. Understanding age, gender and nationality of authors, may answer questions related to diversity of the speakers. This data could be parcially scrapped from Wikipedia as there is a dedicated website that tracks this information. 
    https://en.wikipedia.org/wiki/List_of_TED_speakers